<a href="https://colab.research.google.com/github/hieunguyen7337/LLM_RL/blob/main/hangman_game_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from trl import GRPOTrainer

In [ ]:
# model_name = "Qwen/Qwen3-0.6B"
model_name = "Qwen/Qwen2-0.5B-Instruct"

In [ ]:
dataset = load_dataset("trl-lib/tldr", split="train")

In [ ]:
# Dummy reward function: count the number of unique characters in the completions
def reward_num_unique_chars(completions, **kwargs):
    return [len(set(c)) for c in completions]

In [ ]:
trainer = GRPOTrainer(
    model=model_name,
    reward_funcs=reward_num_unique_chars,
    train_dataset=dataset,
)

In [ ]:
trainer.train()